In [1]:
#预定义
from sklearn import tree               #导入模型
from sklearn.datasets import load_wine        #加载红酒数据  共178个样本  代表了红酒的三个档次（分别有59,71,48个样本） 以及与之对应的13维的属性数据 
from sklearn.model_selection import train_test_split#制作数据集和测试集
import pandas as pd



In [4]:
wine = load_wine()
print(wine.data.shape)#形状 （178,13）
print(wine.target)#（178，）
pd.concat([pd.DataFrame(wine.data),pd.DataFrame(wine.target)],axis=1)
print(wine.feature_names)
print(wine.target_names)

(178, 13)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
['class_0' 'class_1' 'class_2']


In [5]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(wine.data,wine.target,test_size=0.3)#分训练集、测试集  测试集占0.3
print(Xtrain.shape)#训练集形状
print(Xtest.shape)#测试集形状   

(124, 13)
(54, 13)


In [6]:
clf = tree.DecisionTreeClassifier(criterion="entropy")# 载入决策树分类模型
clf = clf.fit(Xtrain, Ytrain)# 决策树拟合，得到模型
score = clf.score(Xtest, Ytest) #返回预测的准确度
print(score)

0.9629629629629629


In [10]:
feature_name = ['酒精','苹果酸','灰','灰的碱性','镁','总酚','类黄酮','非黄烷类酚类','花青素','颜色强度','色调','od280/od315稀释葡萄酒','脯氨酸']


In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf,feature_names= feature_name,class_names=["琴酒","雪莉","贝尔摩德"],filled=True,rounded=True)
graph = graphviz.Source(dot_data)#画树
graph

In [11]:
#特征重要性
print(clf.feature_importances_)
print([*zip(feature_name,clf.feature_importances_)])

[0.         0.         0.         0.03403817 0.         0.
 0.39855519 0.         0.         0.26134164 0.         0.
 0.306065  ]
[('酒精', 0.0), ('苹果酸', 0.0), ('灰', 0.0), ('灰的碱性', 0.034038168143105924), ('镁', 0.0), ('总酚', 0.0), ('类黄酮', 0.39855518847587224), ('非黄烷类酚类', 0.0), ('花青素', 0.0), ('颜色强度', 0.26134163938033883), ('色调', 0.0), ('od280/od315稀释葡萄酒', 0.0), ('脯氨酸', 0.30606500400068304)]


In [36]:
import numpy as np
print(type(np.nonzero([1])))
print(type(np.nonzero([1])[0]))


<class 'tuple'>
<class 'numpy.ndarray'>


In [39]:
type(np.inf)

float

In [ ]:

import numpy
def refreshWeightArr(expon, original_weightArr):
    '''
    这是一个训练数据集的权值分布WeightArr的更新函数
    输入：expon是由基分类器的权重（由分类错误率计算得到的）、类别标签以及单层决策树划分后得到的类别估计值计算得到的
         original_weightArr是更新之前的WeightArr
    输出：updated_weightArr是更新之后的训练数据集的权值分布
    '''
    intermediate_variable = original_weightArr * numpy.exp(expon)
    updated_weightArr = intermediate_variable / intermediate_variable.sum()
    return updated_weightArr
 
 
def adaBoostTrain(dataArr, labelArr, iterationsNumber=40):
    '''
    这是基于单层决策树的adaBoost算法
    输入：dataArr是numpy.ndarray形式的样本数据集
          labelArr是numpy.ndarray形式的类别标签
          iterationsNumber是迭代次数
    输出：多个弱分类器组成的numpy.ndarray形式的分类器集合
    '''
    classifier_Arr = []
    m = dataArr.shape[0]
    weightArr = numpy.ones((m, 1)) / m
    accumulative_estimated_value = numpy.zeros((m, 1))
    for i in range(iterationsNumber):
        current_SigleTree, current_Error, current_label_Result =\
                    buildSingleTree(dataArr, labelArr, weightArr)
        alpha = float(0.5 * numpy.log((1.0 - current_Error) / max(current_Error, 1e-16)))
        current_SigleTree['alpha'] = alpha
        classifier_Arr.append(current_SigleTree)
        expon = -1 * alpha * numpy.array(labelArr).reshape(m, 1) * current_label_Result
        weightArr = refreshWeightArr(expon, weightArr)
        accumulative_estimated_value += alpha * current_label_Result
        if len(numpy.nonzero(numpy.sign(accumulative_estimated_value) !=\   
                    labelArr.reshape(m, 1))[0]) == 0:
            break
 
    return classifier_Arr

In [1]:
import numpy as np

feature = np.array([1,0,-1,2,22])
threshold = 1
op = 0
y_pre = 1*(feature >= threshold) if op == 1 else 1*(feature < threshold)
y_pre

array([0, 1, 1, 0, 0])

In [8]:
a = np.empty((2,2))
print(a)
print(a[:,0])
print(a[:,1])

[[2.12199579e-314 6.95187041e-310]
 [3.57703528e-321 1.11830007e-311]]
[2.12199579e-314 3.57703528e-321]
[6.95187041e-310 1.11830007e-311]
